In [1]:
from itertools import count

import pandas as pd
import csv


import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import statsmodels.api as sm

import statsmodels.formula.api as smf


from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
#read Houseprices data
adv = pd.read_csv(r"HousePrices.csv")


#view first 5 rows of the data
adv[:5]

,price,lotsize,bedrooms,bathrooms,stories,driveway,recreation,fullbase,gasheat,aircon,garage,prefer
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [5]:
# Shows the first five rows of the data
adv.head()


,price,lotsize,bedrooms,bathrooms,stories,driveway,recreation,fullbase,gasheat,aircon,garage,prefer
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [7]:
#Checking the number of rows and columns
adv.shape

(546, 12)

In [9]:
#To see the statistical details of the dataset, we can use describe():
adv.describe()

,price,lotsize,bedrooms,bathrooms,stories,garage
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,68121.597070,5150.265568,2.965201,1.285714,1.807692,0.692308
std,26702.670926,2168.158725,0.737388,0.502158,0.868203,0.861307
min,25000.000000,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,49125.000000,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,62000.000000,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,82000.000000,6360.000000,3.000000,2.000000,2.000000,1.000000
max,190000.000000,16200.000000,6.000000,4.000000,4.000000,3.000000


In [ ]:
#Part 1 - ii
#page 126 of textbook

i.	Construct a summary stat for all the variables in the HousePrices data. (provide Mean, Median, Max, Min, and Std Dev of the data for all variables) (4 points)

In [6]:
adv.describe()

,price,lotsize,bedrooms,bathrooms,stories,garage
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,68121.597070,5150.265568,2.965201,1.285714,1.807692,0.692308
std,26702.670926,2168.158725,0.737388,0.502158,0.868203,0.861307
min,25000.000000,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,49125.000000,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,62000.000000,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,82000.000000,6360.000000,3.000000,2.000000,2.000000,1.000000
max,190000.000000,16200.000000,6.000000,4.000000,4.000000,3.000000


ii.	What is the percentage of houses in the data with Driveway, Gas-Heat and Air-conditioning present?  (Hint: find the mean after creating dummy variables with driveway, gasheat, and aircon variables respectively).

In [3]:
adv_length = len(adv)
adv_filtered_df = adv[(adv['driveway'] == 'yes') & (adv['aircon'] == 'yes') & (adv['gasheat'] == 'yes')]
adv_filtered_df_len = len(adv_filtered_df)
adv_percentage = (adv_filtered_df_len / adv_length) * 100
adv_percentage_rounded = round(adv_percentage, 2)
print(f'The percentage is {adv_percentage_rounded} ')
#print(adv)

The percentage is 0.18 


iii.	Construct a linear regression model to test whether number of bedrooms influence house prices.  Provide a summary of the linear regression model using summary() function.  			(5 points)

In [4]:
X = sm.add_constant(adv['bedrooms']) 
model = sm.OLS(adv['price'], X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     84.38
Date:                Sat, 15 Feb 2025   Prob (F-statistic):           8.53e-19
Time:                        11:44:21   Log-Likelihood:                -6300.0
No. Observations:                 546   AIC:                         1.260e+04
Df Residuals:                     544   BIC:                         1.261e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.877e+04   4413.753      6.519      0.0

iv.	Construct a multiple linear regression model by including all variables as predictors of house prices (response variable) and observe the effect on the house prices. Provide a summary of the regression model using summary() function.  (5 points)

In [5]:
def map_function(x):
    return 1 if x else 0

columns_to_convert = ['driveway', 'recreation', 'fullbase', 'gasheat', 'aircon', 'prefer']
adv2 = adv
adv2[columns_to_convert] = adv2[columns_to_convert].map(map_function)

X = sm.add_constant(adv[['lotsize', 'bedrooms', 'bathrooms', 'stories', 'driveway', 'recreation', 'fullbase', 'gasheat', 'aircon', 'garage', 'prefer']])
model = sm.OLS(adv['price'], X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.557
Method:                 Least Squares   F-statistic:                     138.2
Date:                Sat, 15 Feb 2025   Prob (F-statistic):           3.38e-94
Time:                        11:44:25   Log-Likelihood:                -6114.4
No. Observations:                 546   AIC:                         1.224e+04
Df Residuals:                     540   BIC:                         1.227e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lotsize        4.7334      0.380     12.462      0.0

Part 2: 		(20 points)
Use the Credit data to perform the following tests using Linear Regression settings:
A.	Perform the following steps: 	(5 points) 
i.	Observe the dimension of the Credit data.	(1 points)	
ii.	Provide a summary stat for the variables in Credit data.		(1 points)
iii.	What is the percentage of Student in the Credit data? What is the percentage of Female in the Credit data? What is the percentage of Student who are Female in the Credit data?	(3 points)


In [3]:
df = pd.read_csv(r"Credit.csv")
print(df.shape)
print(df.describe())
studentpct = (df['Student'] == 'Yes').sum()/400
print("the percentage of students is:",studentpct)
femalepct = (df['Gender'] == 'Female').sum()/400
print ("The percent of users identifying as female is:", femalepct)

(400, 12)
       Unnamed: 0      Income         Limit      Rating       Cards  \
count  400.000000  400.000000    400.000000  400.000000  400.000000   
mean   200.500000   45.218885   4735.600000  354.940000    2.957500   
std    115.614301   35.244273   2308.198848  154.724143    1.371275   
min      1.000000   10.354000    855.000000   93.000000    1.000000   
25%    100.750000   21.007250   3088.000000  247.250000    2.000000   
50%    200.500000   33.115500   4622.500000  344.000000    3.000000   
75%    300.250000   57.470750   5872.750000  437.250000    4.000000   
max    400.000000  186.634000  13913.000000  982.000000    9.000000   

              Age   Education      Balance  
count  400.000000  400.000000   400.000000  
mean    55.667500   13.450000   520.015000  
std     17.249807    3.125207   459.758877  
min     23.000000    5.000000     0.000000  
25%     41.750000   11.000000    68.750000  
50%     56.000000   14.000000   459.500000  
75%     70.000000   16.000000   863

B.	Construct a linear regression model to test the following: 	(15 points)

Test how Credit Rating and Student effect Credit Card Balance. In the same model, also test whether the effect of Credit Rating on Credit Card Balance is different for students vs. non-students.

Provide a summary of the model. 


Part 3: 		(20 points)
Use the Credit data to perform the following tests using Linear Regression settings: Online quiz will be based on the results of the regressions performed below.


i.	Test whether Age influence Credit Card Balance on the basis of simple linear regression.
(Provide a summary of the model).  (2 points)


ii.	Use Age and Credit Rating as predictors of Credit Card Balance (response variable) in a multiple linear regression setting. (Provide a summary of the model). (2 points)  



iii.	Compare effect of Age from part (i) and (ii): Write the explanation for part (iii) in Python code. (10 points)



iv.	Observe the distribution of Age. Construct 3 dummy variables based on Age distribution: (6 points)
1. Age 40 and below (Age=<40)
2. Age Between 41 to 56 (41=<Age<=56)
3. Age group over 56 (Age>56)

Construct model to observe whether credit card balance is significantly different for different age group. Consider the age group over 56 as the baseline. 


Part 4: 		(40 points)
i.	Download monthly price data of S&P500 and a stock of your choice for the period 01/01/2005 to 12/31/2019 (or any fifteen year period)   					(2 points)


ii.	Compute the monthly returns for the S&P 500 and the stock. Construct one data frame to store the return series. (2 points)